In [ ]:
import scanpy as sc
import pathlib
import igraph
import leidenalg
import numpy as np

In [ ]:
adata_subset = sc.read_h5ad("E9E10NC.AC_neural_and_epi.h5ad")

In [ ]:
# Inspect the data
print(adata_subset)

In [ ]:
# Visualize the clusters
sc.pl.umap(
    adata_subset,
    color='leiden',               # Color by cluster
    legend_loc='on data',         # Display labels directly on the UMAP
    title='Cluster Visualization',  # Add a title
    size=40,                      # Adjust marker size
    alpha=0.1,                    # Adjust marker transparency
    palette='Set2',               # Use a custom color palette
)

In [ ]:
# Load the CSV file
df = pd.read_csv(DATA_DIR/'TREX/E9E10.neural.epi.clone_2_and_more.csv')

# Check the first few rows of the dataframe to understand its structure
print(df.head())


# highlight clone sharing

In [ ]:
# Subset the rows where leiden == 13
clone_to_plot = df[df['leiden'] == 13]

# Extract the cloneid values
cloneid_to_plot = clone_to_plot['cloneid'].unique()

# Check the extracted cloneid values
print(f"Clone IDs: {cloneid_to_plot[:10]}")
print(f"Number of cells to highlight: {len(cloneid_to_plot)}")

In [ ]:
# Find all cells in the dataframe that have any of the cloneid values we extracted
cells_with_cloneid = df[df['cloneid'].isin(cloneid_to_plot)]

# Extract the 'cellid.old' values for these cells
cellid_to_plot = cells_with_cloneid['cellid.old']

# Check the first few cellid values
print(f"Cell IDs to highlight: {cellid_to_plot[:10]}")
print(f"Number of cells to highlight: {len(cellid_to_plot)}")

In [ ]:
# Use the index of the AnnData object to get the matching cells
cells_to_highlight = adata_subset.obs.index[adata_subset.obs.index.isin(cellid_to_plot)]

# Check how many cells are being highlighted
print(f"Number of cells to highlight: {len(cells_to_highlight)}")

In [ ]:
# Create a new categorical column in `adata.obs` to mark cells to highlight
adata_subset.obs['highlight'] = pd.Categorical(adata_subset.obs.index.isin(cells_to_highlight))

# Assign colors for the 'highlight' categories
# 'True' will correspond to the highlighted cells, 'False' to others
adata_subset.uns['highlight_colors'] = ['lightgrey', 'magenta']

# Plot the UMAP, coloring by the 'highlight' column and using the custom size column
sc.pl.umap(adata_subset, 
           color='highlight',  # Color by the new 'highlight' column
           size=50,  # Size according to the 'size' column we created
           add_outline=False,  # Add outline around the points
           title="UMAP with Highlighted Cells",  # Title of the plot
           show=True)  # Show the plot


In [ ]:
# Create a new categorical column in `adata.obs` to mark cells to highlight
adata_subset.obs['highlight'] = pd.Categorical(adata_subset.obs.index.isin(cells_to_highlight))

# Assign colors for the 'highlight' categories
# 'True' will correspond to the highlighted cells, 'False' to others
adata_subset.uns['highlight_colors'] = ['lightgrey', 'magenta']

# Plot the UMAP, coloring by the 'highlight' column
# Note: size is set for all cells, but we will adjust the drawing order manually
sc.pl.umap(adata_subset, 
           color='highlight',  
           size=5,  # Same size for all cells, we control the z-order
           add_outline=False,  # Remove outline
           title="clone sharing",  
           show=False)  # Show=False to control drawing order manually

# Access the matplotlib figure and axis
fig, ax = plt.gcf(), plt.gca()

# Plot the unhighlighted cells (lightgrey) with lower zorder (background)
unhighlighted_coords = adata_subset.obsm['X_umap'][adata_subset.obs['highlight'] == False]
ax.scatter(unhighlighted_coords[:, 0], unhighlighted_coords[:, 1], 
           color='lightgrey',  # Use the lightgrey color
           s=5,  # Set the size for unhighlighted cells
           alpha=0.5,  # Slight transparency for the unhighlighted cells
           zorder=1)  # Lower zorder to place them in the background

# Plot the highlighted cells (magenta) with higher zorder (front)
highlighted_coords = adata_subset.obsm['X_umap'][adata_subset.obs['highlight'] == True]
ax.scatter(highlighted_coords[:, 0], highlighted_coords[:, 1], 
           color='magenta',  # Use the magenta color for highlighted cells
           s=10,  # Set the size for highlighted cells (larger than unhighlighted)
           alpha=0.7,  # Slight transparency for the highlighted cells
           zorder=2)  # Higher zorder to place them in the front

